In [1]:
## I am going to use the swagger API client for cbioportal to generate some basic folders structure
## for the data set
import girder_client
dccGC = girder_client.GirderClient(apiUrl="https://imaging.htan.dev/girder/api/v1")
from bravado.client import SwaggerClient
cbioportal = SwaggerClient.from_url('https://www.cbioportal.org/api/api-docs',
                                    config={"validate_requests":False,"validate_responses":False})
#https://docs.cbioportal.org/6.-web-api-and-clients/api-and-api-clients

In [2]:
dir(cbioportal)

['Cancer_Types',
 'Clinical_Attributes',
 'Clinical_Data',
 'Clinical_Events',
 'Copy_Number_Segments',
 'Discrete_Copy_Number_Alterations',
 'Gene_Panels',
 'Generic_Assays',
 'Genes',
 'Molecular_Data',
 'Molecular_Profiles',
 'Mutations',
 'Patients',
 'Reference_Genome_Genes',
 'Resource_Data',
 'Resource_Definitions',
 'Sample_Lists',
 'Samples',
 'Structural_Variants',
 'Studies',
 'Treatments']

In [3]:
for a in dir(cbioportal):
    cbioportal.__setattr__(a.replace(' ', '_').lower(), cbioportal.__getattr__(a))

In [4]:
for ct in cbioportal.Cancer_Types.getAllCancerTypesUsingGET().result():
    print(ct)

TypeOfCancer(cancerTypeId='aa', clinicalTrialKeywords='aggressive angiomyxoma', dedicatedColor='LightYellow', name='Aggressive Angiomyxoma', parent='soft_tissue', shortName='AA')
TypeOfCancer(cancerTypeId='aastr', clinicalTrialKeywords='anaplastic astrocytoma', dedicatedColor='Gray', name='Anaplastic Astrocytoma', parent='difg', shortName='AASTR')
TypeOfCancer(cancerTypeId='abc', clinicalTrialKeywords='activated b-cell type', dedicatedColor='LimeGreen', name='Activated B-cell Type', parent='dlbclnos', shortName='ABC')
TypeOfCancer(cancerTypeId='abl', clinicalTrialKeywords='acute basophilic leukemia', dedicatedColor='LightSalmon', name='Acute Basophilic Leukemia', parent='amlnos', shortName='ABL')
TypeOfCancer(cancerTypeId='aca', clinicalTrialKeywords='adrenocortical adenoma', dedicatedColor='Purple', name='Adrenocortical Adenoma', parent='adrenal_gland', shortName='ACA')
TypeOfCancer(cancerTypeId='acbc', clinicalTrialKeywords='adenoid cystic breast cancer', dedicatedColor='HotPink', na

TypeOfCancer(cancerTypeId='thpa', clinicalTrialKeywords='papillary thyroid cancer', dedicatedColor='Teal', name='Papillary Thyroid Cancer', parent='wdtc', shortName='THPA')
TypeOfCancer(cancerTypeId='thpd', clinicalTrialKeywords='poorly differentiated thyroid cancer', dedicatedColor='Teal', name='Poorly Differentiated Thyroid Cancer', parent='thyroid', shortName='THPD')
TypeOfCancer(cancerTypeId='thrlbcl', clinicalTrialKeywords='t-cell/histiocyte-rich large b-cell lymphoma', dedicatedColor='LimeGreen', name='T-Cell/Histiocyte-Rich Large B-Cell Lymphoma', parent='mbn', shortName='THRLBCL')
TypeOfCancer(cancerTypeId='thyc', clinicalTrialKeywords='thymic carcinoma', dedicatedColor='Purple', name='Thymic Carcinoma', parent='tet', shortName='THYC')
TypeOfCancer(cancerTypeId='thym', clinicalTrialKeywords='thymoma', dedicatedColor='Purple', name='Thymoma', parent='tet', shortName='THYM')
TypeOfCancer(cancerTypeId='thymus', clinicalTrialKeywords='thymus', dedicatedColor='Purple', name='Thymus'

In [5]:
tcgaStudyIdList = []

for sl in cbioportal.Sample_Lists.getAllSampleListsUsingGET().result():
    if sl.category == 'all_cases_in_study' and 'tcga' in sl.studyId:
        ### Going to do a subfilter as well... I want all the samples not just the ones used in the pub
        if 'pub' not in sl.studyId:  ## Other logic is to use ends_with "all"
            tcgaStudyIdList.append(sl.studyId)
        
## Want category=='all_cases_in_study'

In [6]:
## These are the main tcga groups not subdivided based on publications and such
tcgaRootCancerTypes = ['acc_tcga', 'cesc_tcga', 'chol_tcga','esca_tcga', 'gbm_tcga','hnsc_tcga',
 'kirp_tcga', 'lusc_tcga', 'meso_tcga','thym_tcga', 'thca_tcga', 'ucs_tcga', 'uvm_tcga',
 'skcm_tcga', 'paad_tcga', 'pcpg_tcga', 'prad_tcga', 'stad_tcga', 'tgct_tcga', 'kirc_tcga', 'dlbc_tcga', 
 'lihc_tcga', 'luad_tcga', 'ov_tcga', 'ucec_tcga', 'brca_tcga', 'kich_tcga']

In [8]:
## I can add an offest if I want to this call


import sys
class LinePrinter():
    """
    Print things to stdout on one line dynamically
    """
    def __init__(self,data):
        sys.stdout.write("\r\x1b[K"+data.__str__())
        sys.stdout.flush()
        
        
def getAllItems( client, folderOrCollectionId, parentType='folder', batchSize = 1000, offset=0, debug=False ):
    # client is the girder client Object
    # I am making this more generic, so I can pass a folder or collection Id
    # This now returns a generator so should be more efficient
#     itemSet = []
    limit = batchSize
    batchesProcessed = 0
    colId = folderOrCollectionId

    getStr = "resource/%s/items?limit=%d&offset=%d&type=%s" % (colId,limit,offset,parentType) 
    #if debug: print(getStr)
    gcImageList = client.get( getStr)

    while gcImageList:
        for i in gcImageList:
            yield(i)

        
        batchesProcessed +=1
#         itemSet.extend(gcImageList)

        getStr = "resource/%s/items?limit=%d&offset=%d&type=%s" % ( colId,batchSize,(int(batchSize)*int(batchesProcessed)),parentType) 
        if debug:
            output = "Processed %d items" % (int(batchSize)*int(batchesProcessed))
            LinePrinter(output)
        gcImageList = client.get( getStr)
        

        

In [7]:
import tqdm
import re
import girder_utils as gu
tcga2OpenFolderId = '5fbc102f79c054aa9327869e'
matchTCGAsvs = re.compile(r'(?P<tcgaID>TCGA-..-....).*(?P<ext>svs)')


tcgaSubjFolderCache = {}
dccGC.authenticate(interactive=True)

Login or email: admin
Password for admin: ········


{'_accessLevel': 2,
 '_id': '5ebee66965f992910fa19570',
 '_modelType': 'user',
 'admin': True,
 'created': '2020-05-15T18:58:49.562000+00:00',
 'email': 'girder.admin@kitware.com',
 'emailVerified': True,
 'firstName': 'admin',
 'groupInvites': [],
 'groups': [],
 'lastName': 'admin',
 'login': 'admin',
 'otp': False,
 'public': True,
 'size': 167707610500,
 'status': 'enabled'}

In [ ]:
tgtTCGARootFolder = '5fc2f483a852689f133b147d'

#for i in getAltqdm.tqdm(dccGC.listResource('resource/%s/items?type=folder' % tcga2OpenFolderId)):
for i in getAllItems(dccGC,tcga2OpenFolderId,offset=81000,debug=True):
    m = matchTCGAsvs.search(i['name'])
    if m:
        tcgaID = m.groupdict()['tcgaID']
        if tcgaID not in tcgaSubjFolderCache:
            tcgaSubjFolder = dccGC.createFolder(tgtTCGARootFolder,tcgaID,reuseExisting=True)
            tcgaSubjFolderCache[tcgaID] = tcgaSubjFolder
    
        tcgaSubjFolder = tcgaSubjFolderCache[tcgaID]    
        ### ADD A CHECK TO AVOID COPYING FILES THAT ARE ALREADY IN THERE

        itemsInFolder = list( dccGC.listItem(tcgaSubjFolder['_id']))
        
        if i['name'] not in [x['name'] for x in itemsInFolder]:
            dccGC.post(f"/item/{i['_id']}/copy?folderId={tcgaSubjFolder['_id']}")

Processed 69000 items